In [17]:
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
import os
import json
import numpy as np

In [2]:
%%time
df_2022 = pd.read_csv('../전처리/wi-fi/path/presence_path_2022.csv', encoding='cp949')

CPU times: total: 30.1 s
Wall time: 32 s


In [3]:
# 용량 줄이기
# 데이터 범위 체크 후, 최적화된 타입 적용
def reduce_mem_usage(df):
    
    # 현재 메모리 사용량
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터 프레임의 메모리 사용량 : {:.2f} MB'.format(start_mem))
    
    
    # 컬럼 하나씩 확인 후 적용
    for col in tqdm(df.columns[1:]):
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
            
    # 타입변경 후 메모리 사용량        
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 메모리 사용량 : {:.2f} MB'.format(end_mem))
    print('감소율 : {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [4]:
# 데이터프레임 타입변경
def df_astype(df):
    reduce_mem_usage(df)
    df['STARTTIMESTAMP'] = pd.to_datetime(df['STARTTIMESTAMP'])
    df['ENDTIMESTAMP'] = pd.to_datetime(df['ENDTIMESTAMP'])

In [5]:
print('df_2022')
df_astype(df_2022)

df_2022
데이터 프레임의 메모리 사용량 : 1765.12 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 826.17 MB
감소율 : 53.2%


In [6]:
df_2022

Path_ID     CLIENTMAC  MAC_add      STARTTIMESTAMP  \
0              472     000000000       61 2022-03-07 14:50:00   
1              473     000000000       37 2022-03-07 15:00:00   
2              474     000000000       37 2022-03-07 15:10:00   
3              475     000000000       34 2022-03-07 17:20:00   
4              476     000000000       34 2022-03-07 17:30:00   
...            ...           ...      ...                 ...   
28919716  73253299  FDF9ED2692B3       32 2022-09-04 20:50:00   
28919717  73253360  FDFBDF7CAD30       37 2022-04-22 19:20:00   
28919718  73253365  FDFC03995AB8       32 2022-05-29 18:30:00   
28919719  73253404  FDFD25D887FF       43 2022-03-28 19:00:00   
28919720  73253410  FDFD402243A7       43 2022-04-22 16:50:00   

                ENDTIMESTAMP  Previous_Path  Next_Path  Next_MAC  
0        2022-03-07 15:00:00            NaN      473.0      37.0  
1        2022-03-07 15:10:00          472.0        NaN       NaN  
2        2022-03-07 15:20:00            NaN        NaN       NaN  
3        2022-03-07 17:30:00            NaN        NaN       NaN  
4        2022-03-07 17:40:00            NaN        NaN       NaN  
...                      ...            ...        ...       ...  
28919716 2022-09-04 21:00:00            NaN        NaN       NaN  
28919717 2022-04-22 19:30:00            NaN        NaN       NaN  
28919718 2022-05-29 18:40:00            NaN        NaN       NaN  
28919719 2022-03-28 19:10:00            NaN        NaN       NaN  
28919720 2022-04-22 17:00:00            NaN        NaN       NaN  

[28919721 rows x 8 columns]

In [7]:
# 동일한 위경도값을 보유한 포인트를 1개의 개소로 측정

# 위경도 csv 파일 읽기
wk = pd.read_csv('../전처리/wi-fi/테마형특화단지ap_위경도.csv', encoding='cp949', index_col=['MAC주소'])


# MAC 코드화 딕셔너리 불러오기
with open('../mac_code.json', 'r') as f :
    mac_code = json.load(f, encoding='utf-8')

# key값과 values값 위치 변경
mac_code = dict(zip(mac_code.values(),mac_code.keys()))

In [8]:
# 사용되지 않은 Wi-Fi, 목록에서 제거
for n in wk.index:
    if n not in mac_code.values():
        wk.drop(n, axis=0, inplace=True)

# 위도로 정렬 (같은 위도 묶기)
wk.sort_values('위도', inplace=True)

In [9]:
# 이전 Mac 주소를 담을 변수
f_mac = None

# 동일한 위경도를 보유한 Mac 주소 리스트
repeat = []

# repeat 리스트를 모은 리스트
repeat_all = []

for n in wk.index:
    # 첫행 Skip
    if n == wk.iloc[0].name:
        f_mac = n
        continue
    # 이전 행 (위도, 경도)와 같다면 append
    if (wk.loc[f_mac]['위도'] == wk.loc[n]['위도']) and (wk.loc[f_mac]['경도'] == wk.loc[n]['경도']):
        repeat.append(f_mac)
        repeat.append(n)
    
    # 전체 리스트 
    else:
        # 빈 리스트는 무시
        if len(repeat) == 0:
            f_mac = n
            continue
        repeat_all.append(list(set(repeat)))
        repeat = []
    f_mac = n

In [13]:
with open('../mac_code.json', 'r') as f :
    mac_code = json.load(f, encoding='utf-8')

repeat_num_all = []
repeat_num = []
for n in repeat_all:
    for i in range(len(n)):
        repeat_num.append(mac_code[n[i]])
    repeat_num_all.append(repeat_num)
    repeat_num = []
    
    
# key값과 values값 위치 변경
mac_code = dict(zip(mac_code.values(),mac_code.keys()))


for n in repeat_num_all:
    for i in range(len(n)):
        if i == 0:
            continue
        mac_code[n[i]] = mac_code[n[0]]

In [22]:
# 월별 원본파일
def month_split(DF, year, encoding='cp949'):
    # 년도별 파일의 최대 월 찾기
    length = DF['STARTTIMESTAMP'].dt.month.max()

    # 년도 폴더 생성
    try:
        directory_01 = './전처리/split_원본/' + f'{year}년_월별'
        if not os.path.exists(directory_01):
            os.makedirs(directory_01)
    except OSError:
        print ('Error: Creating directory. ' +  directory_01)
    
    # 월별 나누기
    print('전체 진행도------------------------------------------------------')
    for n in tqdm(range(1, length+1)):
        if n <= 9:
            num = '0' + str(n)
        else:
            num = str(n)

        # 기간설정(월별)
        if n != 12:
            df = DF[(DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-01 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S')) &
                    (DF['STARTTIMESTAMP'] < datetime.strptime(f'{year}-{n+1}-01 00:00:00',
                                                              '%Y-%m-%d %H:%M:%S'))]
        else:
            df = DF[DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{n}-01 00:00:00', 
                                                              '%Y-%m-%d %H:%M:%S')]
        
        
        #----------------------------------------------------------------------------------  
        df.reset_index(drop=True, inplace=True)
        for x in tqdm(range(len(df))):
            df['MAC_add'][x] = mac_code[df['MAC_add'][x]]
            if np.isnan(df['Next_MAC'][x]):
                continue
            else:
                df['Next_MAC'][x] = mac_code[df['Next_MAC'][x]]
        #----------------------------------------------------------------------------------
        
        

        # 월별 csv 파일 저장    
        df.to_csv(f'{directory_01}/pre_{year}_{num}월.csv', encoding=encoding, index=False)

In [23]:
month_split(df_2022, 2022)

전체 진행도------------------------------------------------------


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/4033464 [00:00<?, ?it/s]

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_12040\3541896321.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['MAC_add'][x] = mac_code[df['MAC_add'][x]]


  0%|          | 0/2934074 [00:00<?, ?it/s]

...

  0%|          | 0/2947191 [00:00<?, ?it/s]

  0%|          | 0/2947468 [00:00<?, ?it/s]

  0%|          | 0/2907722 [00:00<?, ?it/s]

  0%|          | 0/2909262 [00:00<?, ?it/s]

  0%|          | 0/3095135 [00:00<?, ?it/s]

# 축제 일자 추출

In [53]:
# 월별 원본파일
def day_split(DF, year, day_1, day_2, name, encoding='cp949'):
    
    # 해당 데이터프레임 월 추출
    df_month = DF['STARTTIMESTAMP'].dt.month.max()
    
    if df_month <= 9:
        df_month = '0' + str(df_month)
    else:
        df_month = str(df_month)
    
    
    
    # 년도 폴더 생성
    try:
        directory_01 = f'./전처리/김해시_축제/축제_원본/{year}년/{df_month}월_{name}'
        if not os.path.exists(directory_01):
            os.makedirs(directory_01)
    except OSError:
        print ('Error: Creating directory. ' +  directory_01)
    
    # 일별 나누기
    for n in tqdm(range(day_1, day_2+1)):

        # 기간설정(일별)
       
        df = DF[(DF['STARTTIMESTAMP'] >= datetime.strptime(f'{year}-{df_month}-{n} 00:00:00',
                                                          '%Y-%m-%d %H:%M:%S')) &
                (DF['STARTTIMESTAMP'] < datetime.strptime(f'{year}-{df_month}-{n+1} 00:00:00',
                                                          '%Y-%m-%d %H:%M:%S'))]
        
        if n <= 9:
            day = '0' + str(n)
        else:
            day = str(n)
        
        
        # 일별 csv 파일 저장    
        df.to_csv(f'{directory_01}/{year}_{df_month}_{day}_{name}.csv', encoding=encoding, index=False)
        
        
        # 일별 폴더 생성
        try:
            directory_02 = directory_01 + f'/{df_month}_{n}_개소별'
            if not os.path.exists(directory_02):
                os.makedirs(directory_02)
        except OSError:
            print ('Error: Creating directory. ' +  directory_02)        
        

        # Wi-Fi 개소 list
        df_mac_li = list(set(df['MAC_add']))
        
        # Wi-Fi 개소별 나누기
        print(f'{n}일 진행도------------------------------------------------------')
        for mac in tqdm(df_mac_li):
            small_df = df[df['MAC_add'] == mac]
            small_df.to_csv(f'{directory_02}/{year}_{df_month}_{n}_{mac}.csv', encoding=encoding, index=False)

### 데이터프레임 생성

In [54]:
df_04m = pd.read_csv('./전처리/split_원본/2022년_월별/pre_2022_04월.csv', encoding='cp949')
df_astype(df_04m)

데이터 프레임의 메모리 사용량 : 69.35 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 33.70 MB
감소율 : 51.4%


In [55]:
df_09m = pd.read_csv('./전처리/split_원본/2022년_월별/pre_2022_09월.csv', encoding='cp949')
df_astype(df_09m)

데이터 프레임의 메모리 사용량 : 177.57 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 83.37 MB
감소율 : 53.0%


In [56]:
df_10m = pd.read_csv('./전처리/split_원본/2022년_월별/pre_2022_10월.csv', encoding='cp949')
df_astype(df_10m)

데이터 프레임의 메모리 사용량 : 188.91 MB


  0%|          | 0/7 [00:00<?, ?it/s]

최적화 후 메모리 사용량 : 88.14 MB
감소율 : 53.3%


### day_split 함수 적용

In [57]:
# day_split(데이터프레임, 년도, 시작일, 종료일, 축제명)
day_split(df_04m, 2022, 23, 24, '김해경로위안민속축제')

  0%|          | 0/2 [00:00<?, ?it/s]

23일 진행도------------------------------------------------------


  0%|          | 0/52 [00:00<?, ?it/s]

24일 진행도------------------------------------------------------


  0%|          | 0/52 [00:00<?, ?it/s]

In [58]:
day_split(df_09m, 2022, 15, 22, '김해예술제')

  0%|          | 0/8 [00:00<?, ?it/s]

15일 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

16일 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

...

  0%|          | 0/53 [00:00<?, ?it/s]

21일 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

22일 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

In [59]:
day_split(df_10m, 2022, 1, 2, '허왕후신행길축제')

  0%|          | 0/2 [00:00<?, ?it/s]

1일 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

2일 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

In [60]:
day_split(df_10m, 2022, 13, 15, '김해문화재야행')

  0%|          | 0/3 [00:00<?, ?it/s]

13일 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

14일 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]

15일 진행도------------------------------------------------------


  0%|          | 0/53 [00:00<?, ?it/s]